# Horoscope generation using Temporal Convolution Networks

The goal of this notebook is to implement a generator of horoscope based on neural networks. 

More specifically, the architecture used is a Temporal Convolution Network based on the research paper ["An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling"](https://arxiv.org/abs/1803.01271). This architecture is fully convolutional and can therefore take arbitrary length sequence as inputs. The main idea of the authors of this paper is to increase the perceptive field of each successive layer using [dilated convolution](https://github.com/vdumoulin/conv_arithmetic). 

The bulk of the code of the TCN has been taken from [the implementation](https://github.com/locuslab/TCN) linked in the article.

The data used here are all the horoscopes published [beliefnet.com](beliefnet.com) for the year 2011. 

The network takes a sequence of `window_size` words as input and outputs a sequence of `window_size` words. The target that we use for training is a slice of the horoscope corresponding to the input slid on step to the right, effectively asking the network what should the next word be. 

## Imports

In [1]:
import random
import math
import itertools

import numpy as np
import pandas as pd

from tqdm import tnrange, tqdm_notebook
from tqdm import tqdm

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim
from torch.nn.utils import weight_norm
from torch.autograd import Variable

## Training parameters

In [2]:
cuda           = True
file_path      = '../data/horoscope_dataset.csv'
window_size    = 100
batch_size     = 90
print_every    = 500
test_seq_size  = 300
epochs         = 60
use_pretrained = False

## Data loading code

In [3]:
def load_data(path, window_size):
    df          = pd.read_csv(path)
    split_texts = [t.lower().split() for t in df.TEXT] 
    text        = list(itertools.chain.from_iterable(split_texts))
    words       = set(text)
    n_words     = len(words)
    idx_to_word = dict(enumerate(words))
    word_to_idx = {word : idx for idx, word in idx_to_word.items()}
    data        = [(text[i : i + window_size], text[i + 1 : i + window_size + 1])
                    for i in range(len(text) - window_size - 1)]

    return n_words, idx_to_word, word_to_idx, data

In [4]:
def encode_seq(seq, word_to_id):
    return [word_to_id[w] for w in seq]

In [5]:
def data_to_tensor(data, word_to_id, n_char):
    input_tensor = torch.LongTensor([encode_seq(input_seq, word_to_id)
                                      for input_seq, _ in data])
    target_tensor = torch.LongTensor([encode_seq(target_seq, word_to_id)
                                       for _, target_seq in data])
    
    return input_tensor, target_tensor

In [6]:
def batch_generator(data, batch_size, n_char, word_to_id, shuffle = True):
    if shuffle:
        data = random.sample(data, len(data))
    
    return (data_to_tensor(data[i : i + batch_size], word_to_id, n_char) 
                 for i in range(0, len(data), batch_size))

## Model visualization code

The model evaluation consists in asking it to generate a long sequence of character. We randomly select an input as a base for our generation and create a new sequence character by character using the model.

In [7]:
def test_model(tcn, final_sequence_size, window_size, n_words, 
               id_to_word, word_to_id, data):
    seq = list(random.choice(data)[0])
    while len(seq) < final_sequence_size:
        # As the sequence is able to take variable length inputs, it could be 
        # interesting to not limit ourselves on inputs of window_size.
        encoded_input = encode_seq(seq[-window_size:], word_to_id)
        input_tensor  = torch.LongTensor([encoded_input])
        X             = Variable(input_tensor)
        X             = X.cuda() if cuda else X 
        y_pred        = tcn(X)
        # It is important to take the maximum on the dim -2 as each channel of 
        # the output will correspond to the score associated to a character.
        word_pred_id  = y_pred.cpu().max(dim = 2)[1][:,-1].data[0]
        word_pred     = id_to_word[word_pred_id]
        seq.append(word_pred)
    
    return ' '.join(seq)

The following function generates an horoscope starting from a `base` supplied by the caller. 

In [8]:
def genererate_long_sequence(tcn, final_sequence_size, n_wordsr, id_to_word, word_to_id, base):
    seq = base.split()

    while len(seq) < final_sequence_size:
        # In this case we do not limit the size of the input to window_size.
        encoded_input = encode_seq(seq, word_to_id)
        input_tensor  = torch.LongTensor([encoded_input])
        X             = Variable(input_tensor)
        X             = X.cuda() if cuda else X 
        y_pred        = tcn(X)
        # It is important to take the maximum on the dim -2 as each channel of 
        # the output will correspond to the score associated to a character.
        word_pred_id  = y_pred.cpu().max(dim = 2)[1][:,-1].data[0]
        word_pred     = id_to_word[word_pred_id]
        seq.append(word_pred)
        
    return ' '.join(seq)

## Model definition

The `TransposeLayer` is just a simple transposition that reformats the output of the embedding layer into the correct format for the convolution layers.

In [9]:
class TransposeLayer(nn.Module):
    def forward(self, x):
        return x.transpose(-2, -1)

The `Chomp1d` module is used to remove the extra values at the end of the sequence by the padding of a convolution. As the TCN architecture uses dilated convolution, the padding have to be increased in order to be able to generate a long enough output. We have to remove the extra values so that the last value of our output is the result of a dilated convolution whose rightmost value was the last value of the input sequence. 

In [10]:
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size
        
    def forward(self, x):
        # As x can be stored on the GPU, if we use it to build a new tensor,
        # we have to ensure that our new value is stored contiguously.
        return x[:, :, :-self.chomp_size].contiguous()

The `TemporalBlock` module is a residual block containing two weight normalized dilated convolutions with ReLU activations and dropout2d (we drop whole channel at once). The residual connection may contain a 1x1 convolution if it is necessary to transform the input to the correct number of channels. 

In [11]:
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout = 0.2):
        super(TemporalBlock, self).__init__()
        conv_params = {
            'kernel_size' : kernel_size,
            'stride'      : stride,
            'padding'     : padding,
            'dilation'    : dilation
        }
        self.conv1    = weight_norm(nn.Conv1d(n_inputs, n_outputs, **conv_params))
        self.chomp1   = Chomp1d(padding)
        self.relu1    = nn.ReLU()
        self.dropout1 = nn.Dropout2d(dropout)
        self.conv2    = weight_norm(nn.Conv1d(n_outputs, n_outputs, **conv_params))
        self.chomp2   = Chomp1d(padding)
        self.relu2    = nn.ReLU()
        self.dropout2 = nn.Dropout2d(dropout)
        self.net      = nn.Sequential(
            self.conv1, 
            self.chomp1,
            self.relu1,
            self.dropout1,
            self.conv2,
            self.chomp2,
            self.relu2,
            self.dropout2
        )
        # If the number of input channels is equal to the number of output channel then
        # no transformation is required.
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu       = nn.ReLU()
        self.init_weights()
        
    def forward(self, x):
        # Convolutional branch of the residual block
        out = self.net(x)
        # Residual branch of the residual block
        res = x if self.downsample is None else self.downsample(x)

        return self.relu(out + res)
    
    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

The Temporal Convolution Network is a sequence of Temporal Blocks whose dilation is doubled at each step. If enough blocks are uses, this definition allows the network to used information for arbitrarily far away in the past to generate its prediction. 

In [12]:
class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, dim_emb, kernel_size, dropout):
        super(TemporalConvNet, self).__init__()
        layers     = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            # The dilation is doubled at each layer to allow an exponential growth of 
            # the receptive field size.   
            dilation_size = 2 ** i
            in_channels   = dim_emb if i == 0 else num_channels[i - 1]
            out_channels  = num_channels[i]
            layers.append(
                TemporalBlock(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride   = 1,
                    dilation = dilation_size,
                    padding  = (kernel_size - 1) * dilation_size,
                    dropout  = dropout
                )
            )
        
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)

In [13]:
class LanguageModel(nn.Module):
    def __init__(self, n_inputs, n_outputs, num_channels, dim_emb = 50, kernel_size = 2, 
                 emb_dropout = 0.1, dropout = 0.2):
        super(LanguageModel, self).__init__()
        self.embedding = nn.Embedding(n_inputs, dim_emb)
        self.tcn = TemporalConvNet(dim_emb, num_channels, dim_emb, kernel_size, dropout)
        self.decoder = nn.Linear(num_channels[-1], n_outputs)
        self.drop = nn.Dropout(emb_dropout)
        self.init_weights()
        
    def init_weights(self):
        self.embedding.weight.data.normal_(0, 0.01)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.normal_(0, 0.01)
        
    def forward(self, input):
        emb = self.drop(self.embedding(input))
        emb = emb.transpose(-2, -1)
        y   = self.tcn(emb)
        y   = y.transpose(-2, -1) 
        y   = self.decoder(y)
        
        return y

## Training

In [14]:
n_words, id_to_word, word_to_id, data = load_data(file_path, window_size)
print(f'number of words: {n_words}')
print(f'size of the dataset: {len(data)}')
num_channels                          = [512] * 8
tcn                                   = LanguageModel(
                                            n_words, 
                                            n_words, 
                                            num_channels, 
                                            emb_dropout = 0.3, 
                                            dropout = 0.5
                                        ) 
tcn                                   = tcn.cuda() if cuda else tcn
# We view the problem as a classification task in which the network tries
# to predict what class the following character should be.   
criterion                             = nn.CrossEntropyLoss()
# We use an Adam optimizer with the default learning rate of 1e-3.
optimizer                             = optim.Adam(tcn.parameters())
tcn

number of words: 34886
size of the dataset: 2767866


LanguageModel(
  (embedding): Embedding(34886, 50)
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv1): Conv1d(50, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp1): Chomp1d(
        )
        (relu1): ReLU()
        (dropout1): Dropout2d(p=0.5)
        (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
        (chomp2): Chomp1d(
        )
        (relu2): ReLU()
        (dropout2): Dropout2d(p=0.5)
        (net): Sequential(
          (0): Conv1d(50, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (1): Chomp1d(
          )
          (2): ReLU()
          (3): Dropout2d(p=0.5)
          (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,))
          (5): Chomp1d(
          )
          (6): ReLU()
          (7): Dropout2d(p=0.5)
        )
        (downsample): Conv1d(50, 512, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv1): Conv

In [15]:
if use_pretrained:
    tcn.load_state_dict(torch.load('../models/tcn_horoscope.h5'))

In [16]:
params = filter(lambda p: p.requires_grad, tcn.parameters())
sum([np.prod(p.size()) for p in params])

27598834

In [17]:
batch_per_epoch  = math.ceil(len(data) / batch_size)
loss_update_rate = 3

In [18]:
for epoch in tnrange(epochs, desc = 'epochs'):
    loss_pbar    = 0 
    running_loss = 0
    generator    = batch_generator(data, batch_size, n_words, word_to_id)

    with tqdm_notebook(
        enumerate(generator), 
        desc = 'batches', 
        total = batch_per_epoch, 
        unit = 'batch '
    ) as pbar:

        for i, (X, y) in pbar:
            X = Variable(X)
            y = Variable(y)
            X = X.cuda() if cuda else X
            y = y.cuda() if cuda else y
            optimizer.zero_grad()
            y_pred = tcn(X)
            reshaped_y      = y.view(-1)
            reshaped_y_pred = y_pred.view(-1, n_words) 
            loss   = criterion(reshaped_y_pred, reshaped_y)
            loss.backward()
            optimizer.step()

            loss_value    = loss.cpu().data[0]
            running_loss += loss_value
            loss_pbar    += loss_value

            if i % loss_update_rate == loss_update_rate - 1:
                pbar.set_postfix(loss = loss_pbar / loss_update_rate)
                loss_pbar = 0

            if i % print_every == print_every - 1:
                test_result = test_model(tcn, test_seq_size, window_size, n_words, 
                                         id_to_word, word_to_id, data)
                tqdm.write(f'Batch: {i + 1 : 6}, '
                           f'loss: {running_loss / print_every : .4f}\n'
                           f'{test_result}\n')
                running_loss = 0
    torch.save(tcn.state_dict(), '../models/tcn_horoscope.h5')

HBox(children=(IntProgress(value=0, description='epochs', max=60), HTML(value='')))

HBox(children=(IntProgress(value=0, description='batches', max=30755), HTML(value='')))

Batch:    500, loss:  6.3791
willing to journey far and wide to find it. fortunately, libraries of information are available online to sate your cravings without having to invest in overseas excursions. this priority shift is thanks to the intellectual aquarius new moon taking center stage in your 9th house of higher education. you may find yourself willing to allocate precious time to focus on learning, even though you're slightly worried that others might deem it foolish. the only person who can really hold you back is yourself, so cut the anchors and elevate your mind. a dissonant chord is struck between your head and you are a best is your current moon of your current current moon of your current current moon of your current moon of your own current moon of your current moon of your current current moon of your current attitude is your current current moon of your current current moon of your current day of your current moon of your current day of your own current moon of your curr

Batch:   3500, loss:  4.1812
but as long as you're going to take a chance, be certain to enjoy yourself every step of the way. the journey, not the destination, is the experience to be savored. you are traveling through familiar territory in the land of relationships today. on one hand, you might have high hopes about someone you admire, but they might not know anything about your feelings. on the other hand, they could be overly assertive as they act out the urgency of the enterprising aries moon. in both cases, your response is the same: walk the road of truth and be willing to take a risk and then be willing to be a bit of a friend or partner and games. however, you might be tempted to make a decision that you are willing to take the high road and then make sure to get started. however, you might not be able to make your dreams come true. you might be tempted to make your dreams come true. you are able to take a risk and then you can be a bit of a martyr and you might not be able to

Batch:   6500, loss:  3.8303
state of mind because your plans are unfolding on schedule now. but something is gnawing away at the edges of your awareness, no matter how upbeat you are feeling. instead of attempting to ignore your pangs of uncertainty, explore the origins of your current doubts. expand your thinking to integrate your fears; once you adjust to the new landscape, you should be able to navigate around the shadows of your subconscious without being afraid of the dark. expressing your feelings today might heat things up more than you expect, so it's a good idea to proceed slowly. unfortunately, suppressing your feelings is not the best way to do the trick. you may be feeling very optimistic about your own feelings today, but you could be able to make a plan for a while longer. you may be feeling as if you are being in a new direction and you don't have to do anything that you are hearing. you could be so busy with your own feelings that you can do whatever you want to do. yo

...

Batch:   3000, loss:  3.0451
your everyday routine grow less important to you now as you seek to break out of any stagnation that lingers in your life. unfortunately, there may not be any quick fixes, for constraining circumstances will take time to untangle. try not to get frustrated if things aren't moving fast enough today. just keep your eyes on your long-term goals and save the small stuff for another day. don't lose faith; change can happen when you least expect it. although you may secretly fear loss of the status quo, you are in an exciting process of gaining freedom as irrepressible uranus squares your key planet venus in your 7th house of relationships. you may be feeling a bit jumbled fish that you initially set aside and you might not be able to get away with your jam-packed schedule. you might be so excited about your future that you miss what you suppress to do in the meantime. reduce the tension by adopting a brazen approach today because you can see the consequences of 

Batch:   6000, loss:  3.0489
liquefies that which is usually solid, making your mind's eye about as accurate as a compass in the bermuda triangle. instead of embarking on a journey that's likely to lead you in circles, ask a friend for a reality-check before making any significant decisions. although you're willing to take on more than your share of the responsibilities, family troubles may be unavoidable now. nevertheless, you might not be able to resolve the discord right away; only time and forgiveness will heal the wounds and soften the scars. unfortunately, your practical calculations are of no use if your logical mind is the only one that nor does not hurry, so you can be so wrapped up in the details that you can do something that is outside of your comfort zone. your current tendency is to take a chance, but you can still be more effective if you don't infuse your time with a risky strategy. you may be feeling a bit nostalgic, longing for a thought-provoking conversation and the

Batch:   9000, loss:  3.0470
much easier to complete the tasks on your list -- one after another -- if they are part of a holistic plan instead of just isolated chores. for the best results, begin slowly and build momentum throughout the day. your determination is your ticket to success. every little thing seems to magically fit into place today, but so much is happening that you can't assimilate all your experiences. you might feel that something isn't quite right, but assume it's just the ebb and flow of your emotions while the evocative moon is back in your changeable sign. allow your shifting visibility gives you the chance to get your message across. but even if you are feeling great, it's still wise to be honest about your intentions before you are ready. you may be pleasantly surprised with the current friction that you want to be in control. however, you may be so quick to do what you can do today that you lose your temper. however, you might not be able to avoid the stormy emo

Batch:  11500, loss:  3.0448
enough courage to ride these emotional tides with awareness and intention, instead of being pulled under by self-doubt. nothing will stop your pursuit of success if only you believe in yourself. you may assume you're in a very delicate position now because your heightened sense of vulnerability tosses you one way or another. you could subconsciously react to the current pressures by protecting yourself and hiding your softer side. unfortunately, this classical defense mechanism only isolates you from loved ones and might even push your closest allies away. on the other hand, you can set the stage for real intimacy and you can see the truth. you may be feeling a bit of excitement today as you attempt to rectify the superficial noise and therefore you can see the source of your own garden.” you might resist the long run to chase the various scenarios again and again, but you must be careful about purchasing the right questions and just coasting later. you mig

KeyboardInterrupt: 

In [19]:
torch.save(tcn.state_dict(), '../models/tcn_horoscope.h5')

In [20]:
genererate_long_sequence(
    tcn, 
    500, 
    n_words, 
    id_to_word, 
    word_to_id, 
    'your day will be bad but you should stay optimistic because'
)

"your day will be bad but you should stay optimistic because you can see the advantages of being crystal clear to your goals. you may be feeling a bit discouraged today, but you still might be tempted to overindulge on your friends and family. however, you may be so concerned with impressing others that you trample over your current role. don't be afraid to take a risk and let your instincts; you'll be able to make it right, even if you are willing to let go of your preferences. protect your instincts and your heart open. you might be feeling a bit of a group today, but you might not be able to avoid the consequences of your current actions. you might be able to get your hands up in the air if you don't take the time to think about it. you may be so eager to be a team player today because you don't want to be able to get your work done. you may be feeling as if you're not sure how to react to the changing circumstances, but you might not be able to make your point. you may be so self-s